# 10. 데크(Deque), 우선순위 큐

## 데크
데크(Deque)는 더블 엔디드 큐의 줄임말로, 글자 그대로 양쪽 끝을 모두 추출할 수 있는, 큐를 일반화한 형태의 추상 자료형(ADT)이다.

데크는 양쪽에서 삭제와 삽입을 모두 처리할 수 있으며, 스택과 큐의 특징을 모두 갖고 있다. 이 추상 자료형의 구현은 배열이나 연결 리스트 모두 가능하지만, 특별히 다음과 같이 이중 연결 리스트로 구현하는 편이 가장 잘 어울인다.


이중 연결 리스트로 구현하게 되면, 양쪽으로 head와 tail이라는 이름의 두 포인터를 갖고 았다가 새로운 아이템이 추가될 때마다 앞쪽 또는 뒤쪽으로 연결 시켜 주기만 하면 된다. 당연히 연결 후에는 포인터를 이동하면 된다.

파이썬은 데크 자료형을 다음과 같이 collecions 모듈에서 deque라는 이름으로 지원한다. 


import collections<br>
d = collections.deque()<br>
type(d)<br> 
<class 'collections.deque'>


이 collections.deque는 이중 연결 리스트로 구현되어 있다. CPython에서는 고정 길이 하위배열을 지닌 이중 연결 리스트로 구현되어 있으며, 내부 구현을 살펴보면 마찬가지로 다음과 같이 구조체인 dequeobject가 block 노드의 이중 연결 리스트로 구현되어 있는 것을 확인할 수 있다. 또한 dequeobject는 왼쪽, 오른쪽 인덱스 정보와 최대 길이 등 여러 가지 부가 정보를 함께 보관하고 있는 풍부한 구조체임을 확인할 수 있다.

```
// cpython/Modules/_collectionsmodule.c
typedef struct BLOCK {
    struct BLOCK *leftlink;
    PyObject *data[BLOCKLEN];
    struct BLOCK *rightlink;
} block;

typedef struct {
    ...
    block *leftblock;
    block *rightblock;
    Py_ssize_t leftindex;
    Py_ssize_t rightindex;
    Py_ssize_t maxlen;
    ...
} dequeobject;
```

6장에서 풀이한 1번 '유효한 팰린드롬' 문제는 문자열의 맨 앞, 맨 뒤 양쪽 끝을 모두 추출해 비교하는 문제로 풀이 #2에서 데크 자료형을 사용해 풀이하는 방식이 얼마나 효율적인지 이미 살펴본 바 있다.

그렇다면 이번에는 이중 연결 리스트를 이용해 데크 자료형을 직접 구현하는 문제를 한번 풀이해보자.

## 26 원형 데크 디자인

다음 연산을 제공하는 원형 데크를 디자인 하라

- MyCircularDeque(k): 데크 사이즈를 k로 지정하는 생성자다.
- insertFront(): 데크 처음에 아이템을 추가하고 성공할 경우 true를 리턴한다.
- insertLast(): 데크 마지막에 아이템을 추가하고 성공할 경우 true를 리턴한다.
- deleteFront(): 데크 처음에 아이템을 삭제하고 성공할 경우 true를 리턴한다.
- deleteLast(): 데크 마지막에 아이템을 삭제하고 성공할 경우 true를 리턴한다.
- getFront(): 데크의 첫 번쨰 아이템을 가져온다. 데크가 비어 있다면 -1을 리턴한다.
- getRear(): 데크의 마지막 아이템을 가져온다. 데크가 비어 있다면 -1을 리턴한다.
- isEmpty(): 데크가 비어 있는지 여부를 판별한다.
- isFull(): 데크가 가득 차 있는지 여부를 판별한다.

### 풀이 1 이중 연결 리스트를 이용한 데크 구현
9장 맨 마지막 문제인 25번 '원형 큐 디자인' 문제에서 원형 큐를 직접 구현해본 바 있다. 이 문제는 원형 데크를 디자인하는 문제다. 데크란 앞서 설명에서 언급했듯이 양쪽 끝을 모두 추출할 수 있는 큐를 말한다. 25번 문제에서는 원형 큐를 배열로 구현했으므로, 이번에는 실제 파이썬의 데크 구현체이기도 한 이중 연결 리스트로 구현해보자, 또한 이 데크 구현 문제의 경우 원형 큐 구현 문제와 달리, 맨 앞에 노드를 추가하는 insertFront() 연산도 있다. 일반적인 배열로는 맨 앞에 요소를 추가하는 작업은 시간 복잡도가 O(n)이기 때문에 구현이 쉽지 않다. 그러나 연결 리스트는 맨 앞에 노드를 추가하는 작업이 그리 어렵지 않다. 먼저 다음과 같이 초기화 함수 `__init__()`을 정의하자.


     def __init__(self, k):
        self.head, self.tail = ListNode(None)
        self.k, self.len = k, 0
        self.head.right, self.tail.left = self.tail, self.head


우리가 구현하려는 데크는 CPython의 구현과 유사하게 왼쪽,오른쪽 인덱스 역할을 하는 head, tail을 정의하고, 최대 길이 정보를 k로 설정한다. 여기에 추가로, guswo rlfdl 정보를 담는 변수가 될 len을 self.len = 0으로 따로 정의해둔다.


다음은 앞쪽에 노드를 추가하는 연산 insertFront()를 구현해보자.

    def insertFront(self, value: int):
        if self.len == self.k:
            return False
        self.len += 1
        self._add(self.head, ListNode(value))
        return True


새로운 노드 삽입시 최대 길이에 도달했을 때는 False를 리턴하고, 이외에는 _add() 메소드를 이용해 head 위치에 노드를 삽입한다. 다음은 뒤쪽에 노드를 추가하는 연산이다.

    def insertLast(self, value: int) -> bool:
        ...
        self._add(self.tail.left,ListNode(value))
        return True

뒤쪽에 추가하는 연산 insertLast()도 마찬가지다. 한 가지 다른 점은 head가 아닌 tail.left에 삽입한다는 점이다. 이제 실제 삽입을 수행하는 내부 함수를 구현해보자. 내부에서만 사용한다는 의미로 PEP 8 명명 규칙 기준에 따라 밑줄(_) 하나로 시작하도록 메소드 명을 다음과 같이 _add()로 정했다.

    def _add(self, node: ListNode, new: ListNode):
        n = node.right
        node.right = new
        new.left, new.right =node, n
        n.left = new

이중 연결 리스트의 삽입 메소드인 _add()는 이 코드에서처럼 여러 단계의 다소 복잡한 구현 과정을 거친다.

이중 연결 리스트에 신규 노드를 삽입하는 _add() 메소드는 이미 있는 노드를 찢어내고 그 사이에 새로운 노드 new를 삽입하는 형태가 된다.

삭제 또한 크게 다르지 않다. 앞쪽 삭제는 head의 노드를, 뒤쪽 삭제는 tail 위치의 노드를 처리한다. 뒤쪽의 경우 다음 코드와 같이, 좀 더 정확히는 tail.left.left를 _del() 메소드로 처리하게 된다

    def deleteFront(self) -> bool:
        ...
        self.len -= 1
        self._del(self.head)

        
    def deleteLast(self) -> bool:
        ...
        self.len -= 1
        self._del(self.tail.left.left)

이처럼 이중 연결 리스트의 삭제는 삽입보다는 비교적 간단한 편이다. 이제 전체 코드를 정리해보면 다음과 같다.